In [18]:
import pandas as pd
import numpy as np

In [ ]:
PATH = "../data/curated"

In [ ]:
df_lp = pd.read_csv(f"{PATH}/lp_imputada.csv")

In [51]:
df_lp['fecha'] = pd.to_datetime(df_lp['fecha'])

In [52]:
df_lp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23376 entries, 0 to 23375
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   fecha    23376 non-null  datetime64[ns]
 1   t_min    23375 non-null  float64       
 2   t_max    23304 non-null  float64       
 3   t_media  23312 non-null  float64       
 4   pp       23305 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 913.3 KB


# Indices de precipitación extrema

In [54]:
def calcular_indices_pp(
    df,
    base_start="1961-01-01",
    base_end="1990-12-31",
    por_estacion=False
):
    df = df.copy()
    df["fecha"] = pd.to_datetime(df["fecha"])
    df = df.set_index("fecha")
    df["pp"] = df["pp"].fillna(0.0)

    # percentiles base
    base = df.loc[base_start:base_end, "pp"]
    p95 = base[base > 1.0].quantile(0.95)
    p99 = base[base > 1.0].quantile(0.99)

    def calc(grp):
        s = grp["pp"]
        return pd.Series({
            "PRCPTOT": s.sum(),
            "SDII": s[s > 1].sum() / max((s > 1).sum(), 1),
            "CWD": _max_consecutive(s, lambda v: v >= 1),
            "CDD": _max_consecutive(s, lambda v: v < 1),
            "R10mm": (s >= 10).sum(),
            "R20mm": (s >= 20).sum(),
            "R40mm": (s >= 40).sum(),
            "R95pTOT": s[s > p95].sum(),
            "R99pTOT": s[s > p99].sum(),
            "Rx1day": s.max(),
            "Rx5day": s.rolling(5).sum().max(),
        })

    if por_estacion:
        df["anio"] = df.index.year
        df["estacion"] = df.index.month.map(estacion_hsur)
        res = df.groupby(["anio", "estacion"]).apply(calc).reset_index()
    else:
        df["anio"] = df.index.year
        res = df.groupby("anio").apply(calc).reset_index()

    return res

def _calcular_indices_por_año(serie, freq, p95, p99):
    """
    Calcula índices ETCCDI para cada año.
    """
    grupos = serie.resample(freq)

    df = pd.DataFrame({
        "PRCPTOT": grupos.sum(),
        "SDII": grupos.apply(lambda x: x[x > 1].sum() / max((x > 1).sum(), 1)),
        "CWD": grupos.apply(lambda x: _max_consecutive(x, cond=lambda v: v >= 1)),
        "CDD": grupos.apply(lambda x: _max_consecutive(x, cond=lambda v: v < 1)),
        "R10mm": grupos.apply(lambda x: (x >= 10).sum()),
        "R20mm": grupos.apply(lambda x: (x >= 20).sum()),
        "R40mm": grupos.apply(lambda x: (x >= 40).sum()),
        "R95pTOT": grupos.apply(lambda x: x[x > p95].sum()),
        "R99pTOT": grupos.apply(lambda x: x[x > p99].sum()),
        "Rx1day": grupos.apply(lambda x: x.max()),
        "Rx5day": grupos.apply(lambda x: x.rolling(5).sum().max())
    })

    df['Año'] = df.index.year
    return df.reset_index(drop=True)


def _calcular_indices_agregado(serie, p95, p99):
    """
    Calcula índices ETCCDI agregados para todo el periodo.
    """
    return pd.DataFrame({
        "PRCPTOT": [serie.sum()],
        "SDII": [serie[serie > 1].sum() / max((serie > 1).sum(), 1)],
        "CWD": [_max_consecutive(serie, cond=lambda v: v >= 1)],
        "CDD": [_max_consecutive(serie, cond=lambda v: v < 1)],
        "R10mm": [(serie >= 10).sum()],
        "R20mm": [(serie >= 20).sum()],
        "R40mm": [(serie >= 40).sum()],
        "R95pTOT": [serie[serie > p95].sum()],
        "R99pTOT": [serie[serie > p99].sum()],
        "Rx1day": [serie.max()],
        "Rx5day": [serie.rolling(5).sum().max()],
    })


def _max_consecutive(serie, cond):
    """
    Calcula la máxima racha consecutiva que cumple una condición.
    """
    max_racha = 0
    racha = 0
    for val in serie:
        if cond(val):
            racha += 1
            max_racha = max(max_racha, racha)
        else:
            racha = 0
    return max_racha

In [56]:
df_pp = df_lp[["fecha", "pp"]]
df_pp_extrema_anual = calcular_indices_pp(df_pp)
df_pp_extrema_estacional = calcular_indices_pp(df_pp, por_estacion=True,)

/tmp/ipython-input-2846002994.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = df.groupby("anio").apply(calc).reset_index()
/tmp/ipython-input-2846002994.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = df.groupby(["anio", "estacion"]).apply(calc).reset_index()


# Calculo indices de temperatura extrema

In [48]:
# --------------------------------------------------------
# UTILIDADES
# --------------------------------------------------------

def rolling_calendar_percentile(series, percentile):
    """
    Calcula percentiles calendáricos usando ventana ±2 días (5 días)
    según metodología ETCCDI. Devuelve una serie alineada por fecha.
    """
    df = pd.DataFrame({"valor": series})
    df["doy"] = df.index.dayofyear

    # duplico años para ventana circular (365 +- 2)
    df2 = pd.concat([df.assign(aux_year=-1),
                     df.assign(aux_year=0),
                     df.assign(aux_year=1)])

    df2["pseudo_doy"] = df2["doy"] + df2["aux_year"] * 365

    percentiles = (
        df2.groupby("doy")
           .apply(lambda g: np.percentile(g["valor"], percentile))
    )

    return series.index.map(lambda d: percentiles[d.dayofyear])


def run_length_condition(arr, condition=True, min_len=6):
    """
    Cuenta el total de días pertenecientes a secuencias de al menos min_len días
    donde arr es True/False.
    """
    count = 0
    total = 0
    for val in arr:
        if val == condition:
            count += 1
        else:
            if count >= min_len:
                total += count
            count = 0
    if count >= min_len:
        total += count
    return total


# --------------------------------------------------------
# CALCULO PRINCIPAL DE ÍNDICES (ANUAL O POR GRUPO)
# --------------------------------------------------------

def compute_indices(df):
    """
    Recibe un DataFrame con un período (año o estación ya filtrada).
    Devuelve un diccionario con todos los índices ETCCDI.
    """

    TX = df["t_max"]
    TN = df["t_min"]
    TG = df["t_med"]

    indices = {}

    # --------------------
    # ÍNDICES SIMPLES
    # --------------------
    indices["FD"] = (TN < 0).sum()
    indices["SU"] = (TX > 25).sum()
    indices["ID"] = (TX < 0).sum()
    indices["TR"] = (TN > 20).sum()

    # --------------------
    # TXx, TNx, TXn, TNn
    # --------------------
    indices["TXx"] = TX.max()
    indices["TNx"] = TN.max()
    indices["TXn"] = TX.min()
    indices["TNn"] = TN.min()

    # --------------------
    # Percentiles del período base
    # --------------------
    TX10 = df["TX10"]
    TX90 = df["TX90"]
    TN10 = df["TN10"]
    TN90 = df["TN90"]

    indices["TN10p"] = (TN < TN10).mean() * 100
    indices["TX10p"] = (TX < TX10).mean() * 100
    indices["TN90p"] = (TN > TN90).mean() * 100
    indices["TX90p"] = (TX > TX90).mean() * 100

    # --------------------
    # WSDI y CSDI (secuencias)
    # --------------------
    indices["WSDI"] = run_length_condition((TX > TX90).values)
    indices["CSDI"] = run_length_condition((TN < TN10).values)

    # --------------------
    # DTR
    # --------------------
    indices["DTR"] = (TX - TN).mean()

    # --------------------
    # GSL (solo si tmed está disponible)
    # --------------------
    if TG.isna().all():
        indices["GSL"] = np.nan
    else:
        above5 = TG > 5
        # primer período >=6 días consecutivos TG>5
        start = None
        count = 0
        for i, v in enumerate(above5):
            if v:
                count += 1
                if count == 6:
                    start = i - 5
                    break
            else:
                count = 0

        if start is None:
            indices["GSL"] = np.nan
        else:
            # después del inicio, primer período >=6 días consecutivos TG<5
            below5 = TG < 5
            count = 0
            end = None
            for j in range(start + 6, len(below5)):
                if below5.iloc[j]:
                    count += 1
                    if count == 6:
                        end = j - 5
                        break
                else:
                    count = 0

            if end is None:
                indices["GSL"] = np.nan
            else:
                indices["GSL"] = end - start

    return indices


# --------------------------------------------------------
# FUNCIÓN PRINCIPAL ANUAL
# --------------------------------------------------------

def calc_indices_temp_anual(df, periodo_base=(1961, 1990)):
    df = df.copy()
    df["fecha"] = pd.to_datetime(df["fecha"])
    df = df.set_index("fecha")

    if "t_med" not in df:
        df["t_med"] = (df["t_max"] + df["t_min"]) / 2

    base = df.loc[str(periodo_base[0]):str(periodo_base[1])]

    TX10 = rolling_calendar_percentile(base["t_max"], 10)
    TX90 = rolling_calendar_percentile(base["t_max"], 90)
    TN10 = rolling_calendar_percentile(base["t_min"], 10)
    TN90 = rolling_calendar_percentile(base["t_min"], 90)

    df["TX10"] = df.index.map(lambda d: TX10[d.dayofyear])
    df["TX90"] = df.index.map(lambda d: TX90[d.dayofyear])
    df["TN10"] = df.index.map(lambda d: TN10[d.dayofyear])
    df["TN90"] = df.index.map(lambda d: TN90[d.dayofyear])

    rows = []
    for anio, g in df.groupby(df.index.year):
        r = compute_indices(g)
        r["anio"] = anio
        rows.append(r)

    return pd.DataFrame(rows)


def estacion_hsur(month):
    if month in [12, 1, 2]:
        return "Verano"
    elif month in [3, 4, 5]:
        return "Otoño"
    elif month in [6, 7, 8]:
        return "Invierno"
    else:
        return "Primavera"


def calc_indices_temp_estacional(df, periodo_base=(1961, 1990)):
    df = df.copy()
    df["fecha"] = pd.to_datetime(df["fecha"])
    df = df.set_index("fecha")

    if "t_med" not in df:
        df["t_med"] = (df["t_max"] + df["t_min"]) / 2

    df["anio"] = df.index.year
    df["estacion"] = df.index.month.map(estacion_hsur)

    base = df.loc[str(periodo_base[0]):str(periodo_base[1])]

    TX10 = rolling_calendar_percentile(base["t_max"], 10)
    TX90 = rolling_calendar_percentile(base["t_max"], 90)
    TN10 = rolling_calendar_percentile(base["t_min"], 10)
    TN90 = rolling_calendar_percentile(base["t_min"], 90)

    df["TX10"] = df.index.map(lambda d: TX10[d.dayofyear])
    df["TX90"] = df.index.map(lambda d: TX90[d.dayofyear])
    df["TN10"] = df.index.map(lambda d: TN10[d.dayofyear])
    df["TN90"] = df.index.map(lambda d: TN90[d.dayofyear])

    rows = []
    for (anio, est), g in df.groupby(["anio", "estacion"]):
        r = compute_indices(g)
        r["anio"] = anio
        r["estacion"] = est
        rows.append(r)

    return pd.DataFrame(rows)


In [60]:
df_temp = df_lp[["fecha", "t_max", "t_min", "t_media"]]
df_temp_extrema_anual = calc_indices_temp_anual(df_lp)
df_temp_extrema_estacional = calc_indices_temp_estacional(df_lp)

/tmp/ipython-input-1961160336.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.percentile(g["valor"], percentile))
/tmp/ipython-input-1961160336.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.percentile(g["valor"], percentile))
/tmp/ipython-input-1961160336.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is d

In [62]:
df_temp_extrema_estacional.head(5)

,FD,SU,ID,TR,TXx,TNx,TXn,TNn,TN10p,TX10p,TN90p,TX90p,WSDI,CSDI,DTR,GSL,anio,estacion
0,15,0,0,0,24.8,15.8,7.8,-3.7,15.217391,8.695652,13.043478,7.608696,0,0,9.672826,NaN,1961,Invierno
1,2,18,0,4,31.1,21.8,13.9,-2.0,14.130435,11.956522,11.956522,10.869565,0,0,10.782609,NaN,1961,Otoño
2,1,26,0,1,31.8,20.2,11.6,-0.2,13.186813,5.494505,15.384615,10.989011,0,0,10.938462,NaN,1961,Primavera
3,0,62,0,10,35.1,23.0,18.1,6.3,17.777778,11.111111,6.666667,4.444444,0,0,11.837778,NaN,1961,Verano
4,14,0,0,0,24.8,16.0,6.6,-3.7,20.652174,17.391304,9.782609,6.521739,0,0,9.683696,NaN,1962,Invierno


In [65]:
df_extremos_anuales = pd.merge(df_pp_extrema_anual, df_temp_extrema_anual, on=["anio"], how="inner")
df_extremos_estacionales = pd.merge(df_pp_extrema_estacional, df_temp_extrema_estacional, on=["anio", "estacion"], how="inner")

In [68]:
df_extremos_anuales.info()
df_extremos_estacionales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   anio     64 non-null     int32  
 1   PRCPTOT  64 non-null     float64
 2   SDII     64 non-null     float64
 3   CWD      64 non-null     float64
 4   CDD      64 non-null     float64
 5   R10mm    64 non-null     float64
 6   R20mm    64 non-null     float64
 7   R40mm    64 non-null     float64
 8   R95pTOT  64 non-null     float64
 9   R99pTOT  64 non-null     float64
 10  Rx1day   64 non-null     float64
 11  Rx5day   64 non-null     float64
 12  FD       64 non-null     int64  
 13  SU       64 non-null     int64  
 14  ID       64 non-null     int64  
 15  TR       64 non-null     int64  
 16  TXx      64 non-null     float64
 17  TNx      64 non-null     float64
 18  TXn      64 non-null     float64
 19  TNn      64 non-null     float64
 20  TN10p    64 non-null     float64
 21  TX10p    64 non-nu

In [69]:
df_extremos_anuales.to_csv("df_extremos_anuales.csv", index=False)
df_extremos_estacionales.to_csv("df_extremos_estacionales.csv", index=False)